# PySpark for Human Resources dataset

In [1]:
sc = SparkContext.getOrCreate()

In [4]:
hrRDD = sc.textFile("../input/HR_comma_sep.csv")
print('The file has %s lines.' % hrRDD.count())

The file has 15000 lines.


In [20]:
header = hrRDD.first()
HR = hrRDD.filter(lambda line: line!=header)\
        .map(lambda line: (line.split(',')[2], float(line.split(',')[0])))\
        .mapValues(lambda value: (value,1))\
        .reduceByKey(lambda x, y: (x[0]+y[0],x[1]+y[1]))\
        .mapValues(lambda value: value[0]/value[1])\
        .map(lambda x: (x[1],x[0]))\
        .sortByKey()\
        .map(lambda x: (x[1],x[0]))

In [25]:
for elem in HR.collect():
    print("Employees who work in {} projects have an average satisfaction of {}".format(elem[0],elem[1]))

Employees who work in 7 projects have an average satisfaction of 0.11871093749999982
Employees who work in 6 projects have an average satisfaction of 0.27345826235093723
Employees who work in 2 projects have an average satisfaction of 0.4787688442211052
Employees who work in 5 projects have an average satisfaction of 0.6788880840275264
Employees who work in 3 projects have an average satisfaction of 0.6876695437731201
Employees who work in 4 projects have an average satisfaction of 0.695131729667812


In [29]:
seqOp = (lambda x, y: (x[0] + y, x[1] + 1))
combOp = (lambda x, y: (x[0] + y[0], x[1] + y[1]))
sc.parallelize([1, 2, 3, 4]).aggregate((0, 0), seqOp, combOp)


(10, 4)

In [31]:
seqOp = (lambda x, y: (x[0] + y, x[1] + 1))
combOp = (lambda x, y: (x[0] + y[0], x[1] + y[1]))
HR = hrRDD.filter(lambda line: line!=header)\
        .map(lambda line: (line.split(',')[2], float(line.split(',')[0])))\
        .aggregateByKey((0, 0), seqOp, combOp)\
        .mapValues(lambda value: value[0]/value[1])\
        .map(lambda x: (x[1],x[0]))\
        .sortByKey()\
        .map(lambda x: (x[1],x[0]))

HR.collect()

[('7', 0.11871093749999982),
 ('6', 0.27345826235093723),
 ('2', 0.4787688442211052),
 ('5', 0.6788880840275264),
 ('3', 0.6876695437731201),
 ('4', 0.695131729667812)]